In [1]:
!git clone https://github.com/mykola-mmm/airbus_kaggel.git

Cloning into 'airbus_kaggel'...
remote: Enumerating objects: 155, done.
remote: Total 155 (delta 0), reused 0 (delta 0), pack-reused 155
Receiving objects: 100% (155/155), 99.97 MiB | 29.27 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [2]:
import os
os.chdir('./airbus_kaggel')


In [3]:
!python --version

Python 3.10.13


In [4]:
!python .\train-model.py --csv_file ./kaggle/input/airbus-kaggle-preprocessed/df.csv --max_number_of_samples 15000 --epochs 100 --batch_size 32 --loss_function dice_bce_loss --patch_size 768

python: can't open file '/kaggle/working/airbus_kaggel/.train-model.py': [Errno 2] No such file or directory
